# INFO 3350/6350

## Lecture 16: Document similarity using document embeddings

## Document similarity using document embeddings

The goal of this lecture is to learn how to use document embeddings in order to explore which texts in a dataset are similar to each other.

This is a follow-up to lecture 14 on word similarity. There, we saw how to use contextual word embeddings to compare meanings of individual words in poems. Here, we will explore what we can do when we are able to represent whole documents using embeddings.

We'll answer two questions:
1. How can we use document embeddings to understand the differences between two categories? As an example, what can we learn about the differences between narrative and non-narrative text?
2. Given a specific document, what are similar documents?

### What's a document embedding?

A **word embedding** represents a word as a vector (i.e. a list of numbers) that can then be compared to other word embeddings. We can use the same idea for documents: a **document embedding** represents an entire document as a vector. Document embeddings can then be compared to each other, just like word embeddings can.

### Models to use for document embeddings

We'll use [Sentence-T5](https://huggingface.co/sentence-transformers/sentence-t5-base) (ST5), an encoder-only variant of the T5 encoder-decoder model that was specifically trained for comparing documents. ST5 works on documents up to 512 tokens long--if a text is longer than this, it will be truncated. This is enough for the datasets in this tutorial, but keep this in mind when working with other datasets.

While this is a starting point, other document embeddings can be a good choice, too, depending on the task. Other options include [General Text Embeddings](https://huggingface.co/thenlper/gte-base) and can be found on Hugging Face's [sentence-transformers](https://huggingface.co/docs/hub/sentence-transformers) package. This is an active area of research, with new models being trained regularly as of late 2024.


## Setup: install and import pacakges

### Setup: choose a runtime with a GPU

### Setup: install and import packages

In [ ]:
# Install packages that we need
!pip install umap-learn

In [ ]:
# Import packages

# For downloading data
import gdown
# For working with JSON files
import json
# For working with LMs
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import numpy as np
import pandas as pd
import operator
# For printing long documents
import textwrap
# For status bars
from tqdm.notebook import tqdm
# For dimensionality reduction
import umap.umap_ as umap
# For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Setup: initialize the ST5 model

In [ ]:
# Initialize ST5 model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/sentence-t5-base')

## Understanding categories with a dataset of narrative

The article [Towards a Data-Driven Theory of Narrativity](https://muse.jhu.edu/pub/1/article/898332/summary) by Andrew Piper and Sunyam Bagga offers a definition of narrative: "Somebody tells someone that someone did something(s) [to someone] somewhere at some time for some reason." Some text contains narrative. Other text, such as scientific papers, does not usually contain narrative. We're going to use document embeddings and similarity to explore this categorization.

**Questions:**

1. How distinct are the categories of narrative and non-narrative text?

2. Given a specific *narrative* text, what do similar documents look like? Given a specific *non-narrative* text, what do similar documents look like?

**Dataset**: We are going to use a dataset of texts that are labeled as either narrative or non-narrative. The [original data](https://doi.org/10.6084/m9.figshare.21656780.v1) accompanied the article "Towards a Data-Driven Theory of Narrativity" by Andrew Piper and Sunyam Bagga in New Literary History, 2023. The code below loads a subset of the original dataset.

**Process**:
Our general strategy first assesses the categories qualitatively and then compares embeddings quantitatively:
1. Load dataset
2. Get document embeddings
3. Visualize document embeddings using dimensionality reduction
4. Use the visualization to identify promising examples to examine more closely
5. For specific examples, use embedding similarity to explore similar documents.

### Download prepared narrativity data

The data is stored as a CSV file with two columns: the first column is the label (either `narrative` or `non-narrative`), and the second column contains the text.

In [ ]:
# First download the file to the Colab runtime
# File without genre information
#url = 'https://drive.google.com/uc?id=1gmVTLcfdzjjpr9P0x7OPjoS1hpu76t0-'
# File with genre information
url = "https://docs.google.com/spreadsheets/d/1HLT8XjlJxGhRJngn03wtZosVaSsaLMhhiJEqMeeASAY/export?format=csv"

filename = 'narrativity.csv'
gdown.download(url, filename, quiet=True)
# Load the data
narrativity_df = pd.read_csv(filename)
# Store the data in lists
narrativity_texts = narrativity_df['text'].tolist()
narrativity_labels = narrativity_df['label'].tolist()

We can view the first several texts and their labels:

In [ ]:
narrativity_df.sample(3)

In [ ]:
print("Narrativity data shape:", narrativity_df.shape)

### Create document embeddings and visualize them with dimensionality reduction

Now that we've downloaded the data, we can construct 768-dimensional document embeddings using ST5. To visualize the similarities between these embeddings, we use UMAP, though PCA might be a viable alternative.

In [ ]:
%%time
# Create the document embeddings by running the texts through ST5
narrativity_embeddings = model.encode(narrativity_texts)

In [ ]:
# Reduce the embeddings to two dimensions for visualization
reducer = umap.UMAP()
narrativity_umap_embeddings = reducer.fit_transform(narrativity_embeddings)

In [ ]:
# This function takes in two-dimensional embeddings and labels
# and draws a scatterplot. Each document's point is colored by its label.
# It also annotates each point with the document's id number
def plot_umap_embeddings(umap_embeddings, labels, annotate_idxs=True):
  df_to_plot = pd.DataFrame({'x': umap_embeddings[:, 0],
                            'y': umap_embeddings[:, 1],
                            'label': labels})
  label_names = list(set(labels))
  label_to_color = {label_names[0]: '#fc8d62',
                    label_names[1]: '#8da0cb'}
  sns.scatterplot(data=df_to_plot, x='x', y='y', hue='label', palette=label_to_color)
  plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
  # Get rid of the axes units and labels
  # The units are not very meaningful when using UMAP
  axis = plt.gca()
  axis.set_xlabel('')
  axis.set_ylabel('')
  axis.set_xticks([])
  axis.set_yticks([])
  if annotate_idxs:
    for idx in range(umap_embeddings.shape[0]):
      plt.annotate(idx, umap_embeddings[idx, :], xytext=umap_embeddings[idx, :]+0.02, fontsize=6, color='k', alpha=0.8)

In [ ]:
# Now plot the embeddings
plot_umap_embeddings(narrativity_umap_embeddings, narrativity_labels)

We can very clearly see that the classes cluster together&mdash;narrative texts (represented as blue dots) are generally on one side of the image and non-narrative texts (orange dots) are on the other. The embeddings capture differences between narrative and non-narrative texts.

Let's look at individual texts now. Each dot is annotated with its index in small gray text. We can see that there are some outlier documents. For example, document 11 is categorized as narrative, but it is surrounded by non-narrative text. What text does this document contain? What do the most-similar documents to it look like?

The following function uses the cosine similarity of the document embeddings to rank all the other embeddings by similarity and print out the three documents that are most similar. (Note that ST5 embeddings are L2-normalized, i.e., their Euclidean length is one, so taking the dot product of any two of them returnms their cosine similarity.)

In [ ]:
# Print out a document and its three closest neighbors
def print_neighbors(text_idx):
  print(f'Query text: {narrativity_labels[text_idx]}')
  print(textwrap.fill(narrativity_texts[text_idx][:512]))
  print()
  # The text embedding is the row in the embeddings matrix
  # with the same id as the document
  this_embedding = narrativity_embeddings[text_idx, :]
  # Get similarities by taking the dot product between the embeddings matrix
  # and the embedding we care about
  cosine_similarities = np.dot(narrativity_embeddings, this_embedding)
  # Sort the similarities in descending order
  ordered_idxs = np.argsort(cosine_similarities)[::-1]
  # Cosine similarity with itself is always near 1
  assert(ordered_idxs[0] == text_idx)
  for num, idx in enumerate(ordered_idxs[1:4]):
    print(f'Nearest neighbor {num}: {narrativity_labels[idx]}, {cosine_similarities[idx]:.4f} cosine similarity')
    print(textwrap.fill(narrativity_texts[idx][:512]))
    print()


In [ ]:
# Print the most similar documents to document 11
print_neighbors(text_idx=11)

Document 11 is a narrative about "the quants of the
philanthropic world". The three most similar documents are all categorized as non-narrative. Perhaps the subject matter of this document is what makes it more similar to these non-narrative texts.

On the other hand, document 39 is a non-narrative text that appears next to many narrative texts in the visualization. Let's examine it and its nearest neighbors:

In [ ]:
print_neighbors(text_idx=39)

We can also look at examples that are solidly narrative and solidly non-narrative. The following text is non-narrative, and it is very similar to other non-narrative texts, as we can see:

In [ ]:
print_neighbors(text_idx=37)

Document 53 is a narrative text that is surrounded in the visualization by other narrative texts. It is a first-person story, as are its nearest neighbors:

In [ ]:
print_neighbors(text_idx=53)

We've now seen how the distinction between narrative and non-narrative text is captured by ST5 document embeddings. We've used embedding similarity to examine outliers in each category as well as more typical examples from each category.

## Poetry dataset

We're going to use a dataset of poems to ask the same questions in another context.
We'll also see some of the limitations of document embeddings.

**Questions:**

1. How distinct are poems from the 19th and 20th centuries?

2. Given a specific poem, what do similar poems look like?

**Dataset**: We are going to use the dataset of c. 30,000 public domain poems used in lecture 15 on word similarities.

First, load the dataset and limit it to poems from the 19th and 20th centuries. We're going to just take the first 400 poems from it so the dataset is small enough for us to quickly get the embeddings.

In [ ]:
url = "https://raw.githubusercontent.com/melaniewalsh/BERT-4-Humanists/main/data/public-domain-poetry.csv"
poetry_df = pd.read_csv(url, encoding='utf-8')

# Only select poems from the 19th and 20th centuries
poetry_df = poetry_df.loc[poetry_df['period'].isin(['19th Century', '20th Century'])]
# Reindex the rows' indices
poetry_df = poetry_df.reset_index(drop=True)
# Subsample poems so it does not take too long to get embeddings
poetry_df = poetry_df.head(400)
# Print the first 10 poems
poetry_df.head(10)

Poems by several authors are in this sample of the poetry dataset. Here's a list of the unique authors:

In [ ]:
set(poetry_df['author'])

Get the ST5 embeddings and perform dimensionality reduction with UMAP:

In [ ]:
poetry_texts = poetry_df['text'].tolist()
poetry_labels = poetry_df['period'].tolist()
# Get the embeddings
poetry_embeddings = model.encode(poetry_texts)
# Run UMAP for dimensionality reduction
reducer = umap.UMAP()
poetry_umap_embeddings = reducer.fit_transform(poetry_embeddings)

Now plot the two-dimensional representation of the embeddings, just like we did with the narrativity dataset:

In [ ]:
# Use the same function we created for plotting above
plot_umap_embeddings(poetry_umap_embeddings, poetry_labels, annotate_idxs=False)

The two categories are much more intermingled here than they were for narrative vs. non-narrative. It's much harder to identify 19th century vs 20th century poems from these embeddings.

We can also explore these poems by choosing a poem and examining which poems are most similar according to the embedding similarity, just like we did to examine the narrative outliers. Let's look at poem 0 and 100, chosen arbitrarily.

In [ ]:
# Print a poem's title and author
def print_poem(df, idx):
  row = df.iloc[idx]
  print(f"{row['title']} by {row['author']} ({row['lifespan']}):")
  print(row['text'].replace('\r\n', ' '))
  print()

# Take as input the index of a poem, and print the nearest neighbors in the
# embedding space.
def print_closest_poems(poem_idx):
  # Get this poem's embedding
  poem_embedding = poetry_embeddings[poem_idx, :]
  # Get cosine similarities with all the other poem embeddings
  cosine_similarities = np.dot(poetry_embeddings, poem_embedding)
  # Sort the poems in descending order by similarity
  ordered_poem_idxs = np.argsort(cosine_similarities)[::-1]
  # A vector's cosine similarity with itself is always 1,
  # so it will be the most similar vector to itself
  assert(ordered_poem_idxs[0] == poem_idx)
  print_poem(poetry_df, poem_idx)
  for num, idx in enumerate(ordered_poem_idxs[1:4]):
    print(f'Nearest neighbor {num}: {poetry_labels[idx]}, {cosine_similarities[idx]:.4f} cosine similarity')
    print_poem(poetry_df, idx)

In [ ]:
# Pick a poem, find closest poems
print_closest_poems(poem_idx=0)

In [ ]:
print_closest_poems(poem_idx=100)

For both of these poems, the other poems with the most similar embeddings are those by the same author! Now, we can't yet be sure why this is the case; we're working with a small dataset that has a *lot* of poems by each of the writers that we examined. But we have a preliminary suggestion that, when representing poems with ST5 document embeddings, authorship may account for a meaningful part of the differences between embeddings. This does not mean that these embeddings are useless, but we (may) have identified an important part of what embedding similarity recovers in this context.

These ST5 embeddings represent author well, while it is much harder to tell poems from different periods apart.